In [1]:
import sys,os,copy,glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats,integrate,optimize,special
sns.set_style('ticks',{'font.family':'Times New Roman', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=2.0)
sns.set_palette(sns.color_palette("Paired"))

### Define number of observed, expected and observed events for separate data taking periods

In [2]:
#nlay6p (numbers refer to the 2017, 2018A and 2018B periods)
nobsList = np.array([6,2,1])
nExpList = np.array([6.7,1.8,5.7])
nExpErrList = np.array([np.sqrt(1.1**2 + 0.7**2),np.sqrt(0.6**2+0.2**2),np.sqrt(1.2**2+0.6**2)])
lumiList = np.array([41.5,21.1,38.6]) #keep the ratio of signals fixed according to the lumi
effList = np.array([(2*11.147e-3+4.5*12.4e-3)/(2+4.5), (2*8e-3+4.5*8.75e-3)/(2+4.5), 
                    (2*6.37e-3+4.5*7.46e-3)/(2+4.5)])

### Compute exact  likelihood

In [3]:
def likelihood(mu,nsig,nExp,nExpErr,nobs):
    """Marginalized likelihood for mu"""
    
    def integrand(nbg):
        nTot = nbg + mu*nsig
        nErr = nExpErr
        p = stats.poisson.pmf(k=int(nobs),mu=nTot)*stats.norm.pdf(x=nbg,loc=nExp,scale=nErr)
        return p
        
    #Marginalize over the background uncertainty
    result = integrate.quad(integrand, 0, max(2*(nobs-nExp),nExp+5*nExpErr))
    
    return result[0]

def p(mu,nsig,nExp,nExpErr,nobs):
    """Integral of the likelihood from zero to mu"""
    
    result = integrate.quad(likelihood, 0, mu,args=(nsig,nExp,nExpErr,nobs,))
    return result[0]

def likelihoodTot(mu,nsigList,nExpList,nExpErrList,nobsList):
    """Compute the total likeghood as product of each individual (marginalized) likelihood"""
    
    ltot = 1.0
    for i,nsig in enumerate(nsigList):
        nExp = nExpList[i]
        nExpErr = nExpErrList[i]
        nobs = int(nobsList[i])
        ltot *= likelihood(mu,nsig,nExp,nExpErr,nobs)
    return ltot

def pTot(mu,nsigList,nExpList,nExpErrList,nobsList):
    """Integral of the likelihood from zero to mu"""
    
    result = integrate.quad(likelihoodTot, 0, mu,args=(nsigList,nExpList,nExpErrList,nobsList,))
    return result[0]

### Compute observed and expected upper limits for each data taking period

In [4]:
for i,nobs in enumerate(nobsList):
    lumi = lumiList[i]
    nsig = 1
    nExp = nExpList[i]
    nExpErr = nExpErrList[i]
    norm = p(20.0,nsig,nExp,nExpErr,nobs)
    ULobs = optimize.brentq(lambda x: p(x,nsig,nExp,nExpErr,nobs)/norm - 0.95,
                        0,20.0)

    normExp = p(20.0,nsig,nExp,nExpErr,nExp)
    ULexp = optimize.brentq(lambda x: p(x,nsig,nExp,nExpErr,int(nExp))/normExp - 0.95,
                        0,20.0)
    
    #In this case ULobs, ULexp refer to the total number of events in each period,
    #rescale by the corresponding limit for the full lumi (for later comparison):
    print(r'Nsig < %1.2f, Nsig (expected) < %1.2f' %(ULobs*sum(lumiList)/lumi,ULexp*sum(lumiList)/lumi)) 

Nsig < 16.56, Nsig (expected) < 16.56
Nsig < 24.31, Nsig (expected) < 19.02
Nsig < 9.28, Nsig (expected) < 16.90


### Compute observed and expected upper limits using the combined likelihood

In [5]:
nsigList = effList*lumiList

norm = pTot(30.0,nsigList,nExpList,nExpErrList,nobsList)
ULobs = optimize.brentq(lambda x: pTot(x,nsigList,nExpList,nExpErrList,nobsList)/norm - 0.95,
                        0,30.0)

normExp = pTot(30.0,nsigList,nExpList,nExpErrList,nExpList)
ULexp = optimize.brentq(lambda x: pTot(x,nsigList,nExpList,nExpErrList,nExpList)/normExp - 0.95,
                        0,30.0)

In [6]:
#In this case ULobs, ULexp refer to the total production cross-section
#Compute the limit on the total number of signal events (summed over the 3 periods):
print(r'Nsig < %1.2f, Nsig (expected) < %1.2f'
     %(ULobs*sum(nsigList),ULexp*sum(nsigList)))

Nsig < 6.85, Nsig (expected) < 8.12


### Compute limit for combined data

In [7]:
#nlay6p:
nobs = nobsList.sum()
nExp = nExpList.sum()
nExpErr = np.sqrt(sum(nExpErrList**2))
nsig = 1 #keep at 1 to get result in number of events

In [8]:
norm = p(30.0,nsig,nExp,nExpErr,nobs)
ULobs = optimize.brentq(lambda x: p(x,nsig,nExp,nExpErr,nobs)/norm - 0.95,
                        0,30.0)

normExp = p(30.0,nsig,nExp,nExpErr,nExp)
ULexp = optimize.brentq(lambda x: p(x,nsig,nExp,nExpErr,nExp)/normExp - 0.95,
                        0,30.0)

In [9]:
#In this case ULobs, ULexp refer to the total number of events
#for the combined data:
print(r'Nsig < %1.2f, Nsig (expected) < %1.2f'
     %(ULobs,ULexp))

Nsig < 6.36, Nsig (expected) < 9.83
